In [63]:
import os
import zipfile
import requests
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import date
from bs4 import BeautifulSoup
import pandas_market_calendars as mcal
from pandas.tseries.offsets import CustomBusinessDay

# Download FTD Data From https://www.sec.gov/data/foiadocsfailsdatahtm (requests)
# Download GME ETF Data From https://www.etf.com/stock/GME (javascript console)
# Convert FTD Zips to csv files
# Retrieve Stock Market Calendar (pandas_market_calendars)
# Build CustomBusinessDay t+3, t+6, t+21, t+35

# Add in gme shares outstanding (ycharts)
# add in gme/other gme etf prices (yfinance)

In [64]:
# Create a directory to save downloaded files
home_directory = os.path.expanduser("~")
os.chdir(f'{home_directory}/Documents')
os.makedirs('Markets', exist_ok=True)
os.chdir(f'{home_directory}/Documents/Markets')
os.makedirs('FTD_Downloads', exist_ok=True)
os.makedirs('FTD_CSVs', exist_ok=True)
os.makedirs('historical_prices', exist_ok=True)

### CREATE MARKET OPEN DATES DATAFRAME

In [65]:
# NYSE Stock Market Days Open
nyse = mcal.get_calendar('NYSE')
# NASDAQ, NYSE, CBOE_Equity_Options, CBOE_Index_Options, Financial_Markets_US

# Show available calendars
print(mcal.get_calendar_names())

mkt_df = nyse.schedule(start_date='2004-01-01', end_date='2025-02-01')
mkt_df['Date'] = mkt_df.index
mkt_df['Date'] = pd.to_datetime(mkt_df['Date'])
mkt_df['Year'] = mkt_df['Date'].dt.year
mkt_df.reset_index(drop=True, inplace=True)
mkt_df[mkt_df['Year']==2024].head(10)


# Identify all business days in the date range of mkt_df
all_business_days = pd.date_range(start=mkt_df['Date'].min(), end=mkt_df['Date'].max(), freq='B')

# Find missing business days (bank holidays)
market_closed_days = all_business_days.difference(mkt_df['Date'])

# Convert missing_days to a list
market_closed_list = market_closed_days.to_list()

tplus35 = CustomBusinessDay(n=35)
tplus35_h = CustomBusinessDay(n=35, holidays=market_closed_list)

# how to add to a date
# df['new_settlement_date'] = df['settlement_date'] + tplus35_h

['ASX', 'BMF', 'CFE', 'CBOE_Futures', 'CBOE_Equity_Options', 'CBOE_Index_Options', 'CME_Equity', 'CBOT_Equity', 'CME_Agriculture', 'CBOT_Agriculture', 'COMEX_Agriculture', 'NYMEX_Agriculture', 'CME_Rate', 'CBOT_Rate', 'CME_InterestRate', 'CBOT_InterestRate', 'CME_Bond', 'CBOT_Bond', 'EUREX', 'HKEX', 'ICE', 'ICEUS', 'NYFE', 'NYSE', 'stock', 'NASDAQ', 'BATS', 'DJIA', 'DOW', 'IEX', 'Investors_Exchange', 'JPX', 'LSE', 'OSE', 'SIFMAUS', 'SIFMA_US', 'Capital_Markets_US', 'Financial_Markets_US', 'Bond_Markets_US', 'SIFMAUK', 'SIFMA_UK', 'Capital_Markets_UK', 'Financial_Markets_UK', 'Bond_Markets_UK', 'SIFMAJP', 'SIFMA_JP', 'Capital_Markets_JP', 'Financial_Markets_JP', 'Bond_Markets_JP', 'SIX', 'SSE', 'TSX', 'TSXV', 'BSE', 'NSE', 'TASE', 'AIXK', 'ASEX', 'BVMF', 'CMES', 'IEPA', 'XAMS', 'XASX', 'XBKK', 'XBOG', 'XBOM', 'XBRU', 'XBSE', 'XBUD', 'XBUE', 'XCBF', 'XCSE', 'XDUB', 'XFRA', 'XETR', 'XHEL', 'XHKG', 'XICE', 'XIDX', 'XIST', 'XJSE', 'XKAR', 'XKLS', 'XKRX', 'XLIM', 'XLIS', 'XLON', 'XMAD', 'XME

### LOAD GME ETF LIST

In [66]:
gme_etf_df = pd.read_csv('gme_etfs.csv', index_col=None)
gme_tickers = gme_etf_df['Ticker'].unique().tolist()
print(len(gme_tickers))
gme_tickers.append('GME')
gme_etf_df.head(5)

# Create ETF DataFrame To See How Many GME Shares Are Held
etf_shares_df = gme_etf_df[['Ticker', 'Shares Held', '% Allocation']].copy()
etf_shares_df.rename(columns={
    'Ticker':'ticker', 
    'Shares Held':'gme_shares_held', 
    '% Allocation':'percent_allocation'
}, inplace=True)
etf_shares_df.head(1)

86


,ticker,gme_shares_held,percent_allocation
0,GAMR,69120,4.38%


### LOAD FTD DATA (ETL FOR CREATING THIS FILE IS BELOW)

In [67]:
ftd_df = pd.read_csv('ftds_gme_related_6_19_24.csv', index_col=0)
ftd_df.rename(columns={
    'SETTLEMENT DATE':'settlement_date', 
    'CUSIP':'cusip', 
    'SYMBOL':'ticker', 
    'QUANTITY (FAILS)':'cumulative_num_fails', 
    'DESCRIPTION':'description', 
    'PRICE':'closing_price_prev_day'
}, inplace=True)
ftd_df['settlement_date'] = pd.to_datetime(ftd_df['settlement_date'], format='%Y%m%d')

ftd_df = ftd_df.merge(etf_shares_df, how='left', on=['ticker'])
ftd_df['percent_allocation'] = ftd_df['percent_allocation'].str.replace('%', '').astype(float)
ftd_df['gme_shares_w_percent'] = (ftd_df['percent_allocation']/100) * ftd_df['cumulative_num_fails']
ftd_df.head(5)

c:\ProgramData\Anaconda3\envs\py37\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,settlement_date,cusip,ticker,cumulative_num_fails,description,closing_price_prev_day,gme_shares_held,percent_allocation,gme_shares_w_percent
0,2013-05-01,25459W417,RETL,359.0,DIREXION DAILY RETAIL BULL 3X,48.59,30630.0,1.88,6.7492
1,2013-05-01,33939L100,TILT,22361.0,FLEXSHARES TR,67.27,11360.0,0.02,4.4722
2,2013-05-01,36467W109,GME,4297.0,GAMESTOP CORP (HLDG CO) CL A,34.90,NaN,NaN,NaN
3,2013-05-01,464287150,ITOT,30135.0,ISHARES CORE S&P TOTAL U.S. ST,72.85,296050.0,0.01,3.0135
4,2013-05-01,464287473,IWS,20983.0,ISHARES RUSSELL MIDCAP VALUE I,57.75,426080.0,0.09,18.8847


In [68]:
gme_ftd_df = ftd_df[ftd_df['ticker']=='GME']
gme_ftd_df = gme_ftd_df[['settlement_date', 'cumulative_num_fails']]
gme_ftd_df.columns = ['date', 'total_gme_ftds']
gme_ftd_df.head(1)

,date,total_gme_ftds
2,2013-05-01,4297.0


In [69]:
etf_fails = ftd_df[ftd_df['ticker']!='GME'].groupby(['settlement_date']).agg({'gme_shares_w_percent':['sum'], 'cumulative_num_fails':['sum']}).reset_index()
etf_fails.columns = ['date', 'total_etf_gme_shares_w_percent_adj', 'total_etf_cumulative_fails']
etf_fails['date'] = pd.to_datetime(etf_fails['date'])
etf_fails.head(5)

,date,total_etf_gme_shares_w_percent_adj,total_etf_cumulative_fails
0,2004-03-22,690.1688,1762282.0
1,2004-03-23,1704.9677,2236476.0
2,2004-03-24,1510.2698,2221507.0
3,2004-03-25,1766.3947,2259678.0
4,2004-03-26,505.5016,1828878.0


### LOAD HISTORICAL PRICES FOR GME AND ETFS (yfinance code below)

In [70]:
all_ticker_df = pd.read_csv('only_gme_related_prices.csv', index_col=0) #filtered from all_gme_related_prices

gme_price_df = all_ticker_df[all_ticker_df['ticker']=='GME'].copy().reset_index(drop=True)
gme_price_df = gme_price_df[['date', 'open', 'high', 'low', 'close', 'volume']]
gme_price_df.rename(columns={
    'open': 'gme_open', 
    'high': 'gme_high', 
    'low': 'gme_low', 
    'close': 'gme_close', 
    'volume': 'gme_volume'
}, inplace=True)
gme_price_df.head(2)

# ticker_df_join = all_ticker_df[all_ticker_df['ticker']!='GME'][['date', 'ticker', 'open', 'high', 'low', 'close', 'volume']].copy().reset_index(drop=True)
# ticker_df_join.columns = ['date', 'ticker', 'ticker_open', 'ticker_high', 'ticker_low', 'ticker_close', 'ticker_volume']

KeyError: 'ticker'

### Load GME Shares Outstanding

In [ ]:
gme_outstanding_shares = pd.read_csv('gme_shares_outstanding.csv', index_col=None)
gme_outstanding_shares.rename(columns={
    'Date': 'date',
    'Ticker': 'ticker',
    'Shares Outstanding': 'gme_shares_outstanding'
}, inplace=True)
gme_outstanding_shares = gme_outstanding_shares[['date', 'gme_shares_outstanding']]
gme_outstanding_shares['date'] = pd.to_datetime(gme_outstanding_shares['date'])
gme_outstanding_shares['date'] = gme_outstanding_shares['date'].dt.date


# Generate date range starting from 2010-03-25 to the end date or present
start_date = date(2010, 3, 25)
end_date = date.today()  # or a specific end date if needed
date_range = pd.date_range(start_date, end_date)

# Create a new DataFrame with the date range
all_dates_df = pd.DataFrame(date_range, columns=['date'])
all_dates_df['date'] = all_dates_df['date'].dt.date

# Merge the new DataFrame with the existing DataFrame
gme_outstanding_all = pd.merge(all_dates_df, gme_outstanding_shares, on='date', how='left')

# Use forward fill to fill missing values in gme_shares_outstanding
gme_outstanding_all['gme_shares_outstanding'].fillna(method='ffill', inplace=True)
gme_outstanding_all['gme_shares_outstanding'] = gme_outstanding_all['gme_shares_outstanding'].astype(int)
gme_outstanding_all.head(2)


,date,gme_shares_outstanding
0,2010-03-25,611300000
1,2010-03-26,611300000


In [ ]:
market_df = mkt_df[['Year', 'Date']].copy()
market_df.columns = ['year', 'date']
market_df = market_df[market_df['year']>=2010].reset_index(drop=True)
market_df = market_df[market_df['date']<='2024-06-18'].reset_index(drop=True)

market_df['date'] = pd.to_datetime(market_df['date'])

# Add GME Price
market_df = market_df.merge(gme_price_df[['date', 'gme_open', 'gme_high', 'gme_low', 'gme_close', 'gme_volume']], how='left', on=['date'])
market_df.head(1)

,year,date,gme_open,gme_high,gme_low,gme_close,gme_volume
0,2010,2010-01-04,3.714934,3.86306,3.703151,3.854644,26702800


In [ ]:
# Add GME Outstanding Shares
gme_outstanding_all['date'] = pd.to_datetime(gme_outstanding_all['date'])
market_df = market_df.merge(gme_outstanding_all, how='left', on=['date'])
market_df.head(1)


,year,date,gme_open,gme_high,gme_low,gme_close,gme_volume,gme_shares_outstanding
0,2010,2010-01-04,3.714934,3.86306,3.703151,3.854644,26702800,NaN


In [ ]:
# ADD FTD Data
gme_ftd_df['date'] = pd.to_datetime(gme_ftd_df['date'])
market_df = market_df.merge(gme_ftd_df, how='left', on=['date'])
market_df.head(1)

,year,date,gme_open,gme_high,gme_low,gme_close,gme_volume,gme_shares_outstanding,total_gme_ftds
0,2010,2010-01-04,3.714934,3.86306,3.703151,3.854644,26702800,NaN,500.0


In [ ]:
# Add ETF Fails Data.
# Add t+3, t+6, t+21, t+35 dates
market_df = market_df.merge(etf_fails, how='left', on=['date'])
market_df.head(1)

,year,date,gme_open,gme_high,gme_low,gme_close,gme_volume,gme_shares_outstanding,total_gme_ftds,total_etf_gme_shares_w_percent_adj,total_etf_cumulative_fails
0,2010,2010-01-04,3.714934,3.86306,3.703151,3.854644,26702800,NaN,500.0,65.9048,160161.0


In [ ]:
# Function to adjust for holidays
def add_days_with_holidays(date, days, holidays):
    current_date = date
    days_added = 0
    
    while days_added < days:
        current_date += pd.Timedelta(days=1)
        if current_date not in holidays:
            days_added += 1
            
    return current_date

In [ ]:
# Add Various t+X dates
for tplus in [3, 6, 21, 30, 31, 32, 33, 34, 35, 36]:
    market_df[f'date_t{tplus}'] = market_df['date'] + pd.Timedelta(days=tplus)
    # market_df[f'date_t{tplus}'] = market_df['date'].apply(lambda x: add_days_with_holidays(x, tplus, market_closed_list))
    # market_df[f'date_t{tplus}'] = market_df['date'] + CustomBusinessDay(n=35, holidays=market_closed_list)

In [71]:
# Add High Price On This t+x Date
for datenum in [3, 6, 21, 30, 31, 32, 33, 34, 35, 36]:
    temp_price_df = gme_price_df[['date', 'gme_high']].copy()
    temp_price_df.columns = [f'date_t{datenum}', f'gme_high_t{datenum}']
    temp_price_df[f'date_t{datenum}'] = pd.to_datetime(temp_price_df[f'date_t{datenum}'])
    market_df = market_df.merge(temp_price_df, how='left', on = [f'date_t{datenum}'])
    print(f'date_t{datenum}')
    
market_df.head(1)

ValueError: You are trying to merge on object and datetime64[ns] columns. If you wish to proceed you should use pd.concat

In [72]:
# Add High Price Increase Percent
for datenum in [3, 6, 21, 30, 31, 32, 33, 34, 35, 36]:
    market_df[f'gme_high_t{datenum}'] = market_df[f'gme_high_t{datenum}'].fillna(0)
    market_df[f'gme_high_percent_increase_t{datenum}'] = (market_df[f'gme_high_t{datenum}'] - market_df['gme_high']) / market_df['gme_high']

market_df.head(1)

,year,date,gme_open,gme_high,gme_low,gme_close,gme_volume,gme_shares_outstanding,total_gme_ftds,total_etf_gme_shares_w_percent_adj,...,gme_high_percent_increase_t32,gme_high_percent_increase_t33,gme_high_percent_increase_t34,gme_high_percent_increase_t35,gme_high_percent_increase_t36,gme_high_t35ish,gme_high_t35ish_plus,gme_high_percent_increase_t35ish,gme_high_percent_increase_t35ish_plus,percent_gme_outstanding_traded
0,2018,2018-01-02,3.922936,3.995018,3.88362,3.988465,11330800,405220000.0,NaN,508.801,...,-1.0,-0.113177,-0.104429,-0.088573,-0.095134,3.641166,3.641166,-0.088573,-0.088573,0.027962


In [73]:
print(market_df.columns.tolist())

['year', 'date', 'gme_open', 'gme_high', 'gme_low', 'gme_close', 'gme_volume', 'gme_shares_outstanding', 'total_gme_ftds', 'total_etf_gme_shares_w_percent_adj', 'total_etf_cumulative_fails', 'date_t3', 'date_t6', 'date_t21', 'date_t30', 'date_t31', 'date_t32', 'date_t33', 'date_t34', 'date_t35', 'date_t36', 'gme_high_t3', 'gme_high_t6', 'gme_high_t21', 'gme_high_t30', 'gme_high_t31', 'gme_high_t32', 'gme_high_t33', 'gme_high_t34', 'gme_high_t35', 'gme_high_t36', 'gme_high_percent_increase_t3', 'gme_high_percent_increase_t6', 'gme_high_percent_increase_t21', 'gme_high_percent_increase_t30', 'gme_high_percent_increase_t31', 'gme_high_percent_increase_t32', 'gme_high_percent_increase_t33', 'gme_high_percent_increase_t34', 'gme_high_percent_increase_t35', 'gme_high_percent_increase_t36', 'gme_high_t35ish', 'gme_high_t35ish_plus', 'gme_high_percent_increase_t35ish', 'gme_high_percent_increase_t35ish_plus', 'percent_gme_outstanding_traded']


In [74]:
market_df['gme_high_t35ish'] = market_df[['gme_high_t33', 'gme_high_t34', 'gme_high_t35', 'gme_high_t36']].max(axis=1)
market_df['gme_high_t35ish_plus'] = market_df[['gme_high_t30', 'gme_high_t31', 'gme_high_t32', 'gme_high_t33', 'gme_high_t34', 'gme_high_t35', 'gme_high_t36']].max(axis=1)

market_df[f'gme_high_percent_increase_t35ish'] = (market_df[f'gme_high_t35ish'] - market_df['gme_high']) / market_df['gme_high']
market_df[f'gme_high_percent_increase_t35ish_plus'] = (market_df[f'gme_high_t35ish_plus'] - market_df['gme_high']) / market_df['gme_high']

In [75]:
market_df['percent_gme_outstanding_traded'] = market_df['gme_volume'] / market_df['gme_shares_outstanding']

In [76]:
market_df = market_df[market_df['date']<='2024-06-18'].reset_index(drop=True)
market_df.to_csv('full_training_file_gme.csv')

### BUILD MODEL

In [77]:
import pandas as pd
market_df = pd.read_csv('full_training_file_gme.csv', index_col=0)

In [78]:
print(market_df.columns.tolist())

['year', 'date', 'gme_open', 'gme_high', 'gme_low', 'gme_close', 'gme_volume', 'gme_shares_outstanding', 'total_gme_ftds', 'total_etf_gme_shares_w_percent_adj', 'total_etf_cumulative_fails', 'date_t3', 'date_t6', 'date_t21', 'date_t30', 'date_t31', 'date_t32', 'date_t33', 'date_t34', 'date_t35', 'date_t36', 'gme_high_t3', 'gme_high_t6', 'gme_high_t21', 'gme_high_t30', 'gme_high_t31', 'gme_high_t32', 'gme_high_t33', 'gme_high_t34', 'gme_high_t35', 'gme_high_t36', 'gme_high_percent_increase_t3', 'gme_high_percent_increase_t6', 'gme_high_percent_increase_t21', 'gme_high_percent_increase_t30', 'gme_high_percent_increase_t31', 'gme_high_percent_increase_t32', 'gme_high_percent_increase_t33', 'gme_high_percent_increase_t34', 'gme_high_percent_increase_t35', 'gme_high_percent_increase_t36', 'gme_high_t35ish', 'gme_high_t35ish_plus', 'gme_high_percent_increase_t35ish', 'gme_high_percent_increase_t35ish_plus', 'percent_gme_outstanding_traded']


In [79]:
trian_cols = ['gme_close', 'gme_volume', 'percent_gme_outstanding_traded', 'total_gme_ftds', 'total_etf_gme_shares_w_percent_adj', 'total_etf_cumulative_fails']

pred_col = 'gme_high_percent_increase_t35ish'
# gme_high_t35, gme_high_t35ish, gme_high_t35ish_plus
# gme_high_percent_increase_t35, gme_high_percent_increase_t35ish, gme_high_percent_increase_t35ish_plus

In [80]:
market_df = market_df[market_df['date']>='2018-01-01'].reset_index(drop=True)

In [81]:
market_df[market_df['date']=='2024-05-17'][['date', 'date_t35', f'{pred_col}']]

,date,date_t35,gme_high_percent_increase_t35ish
1604,2024-05-17,2024-06-23,-1.0


In [82]:
market_df['date'] = pd.to_datetime(market_df['date'])
market_df['date_t35'] = pd.to_datetime(market_df['date_t35'])

In [83]:
import xgboost as xgb
from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

scaler = StandardScaler()

# Splitting the data by date '2022-01-01'
split_date = pd.to_datetime('2022-01-01')

train_df = market_df[(market_df['date'] < split_date)]
test_df = market_df[(market_df['date'] >= split_date) & (market_df['date_t35'] < '2024-06-24')]

# Features and target variable
X_train = train_df[trian_cols].fillna(0)
y_train = train_df[pred_col]
X_test = test_df[trian_cols].fillna(0)
y_test = test_df[pred_col]

# Applying the scaler to the training data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [84]:
# Linear Regression model
linear_rfe = LinearRegression()
rfe_linear = RFE(estimator=linear_rfe, n_features_to_select=6, step=1)
rfe_linear.fit(X_train_scaled, y_train)
y_pred_linear = rfe_linear.predict(X_test_scaled)

# Evaluation
linear_mse = mean_squared_error(y_test, y_pred_linear)
linear_r2 = r2_score(y_test, y_pred_linear)

linear_results = {
    'Linear Regression MSE': linear_mse,
    'Linear Regression R²': linear_r2
}

# XGBoost model
xgb_rfe = xgb.XGBRegressor(objective='reg:squarederror', seed=42)
rfe_xgb = RFE(estimator=xgb_rfe, n_features_to_select=6, step=1)
rfe_xgb.fit(X_train_scaled, y_train)
y_pred_xgb = rfe_xgb.predict(X_test_scaled)

# Evaluation
xgb_mse = mean_squared_error(y_test, y_pred_xgb)
xgb_r2 = r2_score(y_test, y_pred_xgb)

xgb_results = {
    'XGBoost MSE': xgb_mse,
    'XGBoost R²': xgb_r2
}

In [85]:
# Get the selected feature indices and names for both RFE models
linear_selected_features = X_train.columns[rfe_linear.get_support()]
xgb_selected_features = X_train.columns[rfe_xgb.get_support()]

# Print the selected features for both models
linear_selected_features_list = list(linear_selected_features)
xgb_selected_features_list = list(xgb_selected_features)

linear_model = LinearRegression()
# Fit the Linear Regression model with the selected features
linear_model.fit(X_train_scaled[:, rfe_linear.support_], y_train)

# Model details
linear_model_details = {
    'intercept': linear_model.intercept_,
    'coefficients': linear_model.coef_,
    'selected_features': linear_selected_features_list
}

linear_selected_features_list, xgb_selected_features_list, linear_model_details

pred_df = test_df.copy()
pred_df['linear_pred'] = y_pred_linear
pred_df['xgb_pred'] = y_pred_xgb
pred_df['actual'] = pred_df[pred_col]



In [86]:
print(pred_df.columns.tolist())

['year', 'date', 'gme_open', 'gme_high', 'gme_low', 'gme_close', 'gme_volume', 'gme_shares_outstanding', 'total_gme_ftds', 'total_etf_gme_shares_w_percent_adj', 'total_etf_cumulative_fails', 'date_t3', 'date_t6', 'date_t21', 'date_t30', 'date_t31', 'date_t32', 'date_t33', 'date_t34', 'date_t35', 'date_t36', 'gme_high_t3', 'gme_high_t6', 'gme_high_t21', 'gme_high_t30', 'gme_high_t31', 'gme_high_t32', 'gme_high_t33', 'gme_high_t34', 'gme_high_t35', 'gme_high_t36', 'gme_high_percent_increase_t3', 'gme_high_percent_increase_t6', 'gme_high_percent_increase_t21', 'gme_high_percent_increase_t30', 'gme_high_percent_increase_t31', 'gme_high_percent_increase_t32', 'gme_high_percent_increase_t33', 'gme_high_percent_increase_t34', 'gme_high_percent_increase_t35', 'gme_high_percent_increase_t36', 'gme_high_t35ish', 'gme_high_t35ish_plus', 'gme_high_percent_increase_t35ish', 'gme_high_percent_increase_t35ish_plus', 'percent_gme_outstanding_traded', 'linear_pred', 'xgb_pred', 'actual']


In [87]:
out_cols = ['date', 'date_t35', 'gme_high', 'gme_close', 'gme_volume', 'total_gme_ftds', 'total_etf_gme_shares_w_percent_adj', 'total_etf_cumulative_fails',  'gme_high_t35', 'gme_high_t35ish', 'gme_high_percent_increase_t35ish', 'percent_gme_outstanding_traded', 'linear_pred', 'xgb_pred', 'actual']

In [90]:
pred_df[out_cols].to_csv('model_percent_increase_predictions_gme_ftds.csv')

In [89]:

def calculate_closeness(pred_df, pred_col, threshold=0.2):
    # Calculate the number of predictions and actuals below or equal to the threshold
    pred_below_threshold = pred_df[pred_df[pred_col] >= threshold].copy()
    actual_below_threshold = pred_below_threshold[pred_below_threshold['actual'] >= threshold].copy()

    num_predictions = pred_below_threshold.shape[0]
    correct_predictions = actual_below_threshold.shape[0]
    
    # Calculate the percentage of time the predictions are close to the actuals
    if num_predictions > 0:
        percentage_close = (correct_predictions / num_predictions) * 100
    else:
        percentage_close = 0

    return percentage_close, num_predictions

for threshold_num in [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]:
    # threshold_num = 0.2
    linear_percentage_close, linear_num_predictions = calculate_closeness(pred_df, 'linear_pred', threshold_num)
    xgb_percentage_close, xgb_num_predictions = calculate_closeness(pred_df, 'xgb_pred', threshold_num)
    print('\n')
    print(f"Linear Model: {linear_percentage_close:.2f}% of the time, the actual value is >= {threshold_num} when the prediction is >= {threshold_num} (Number of such predictions: {linear_num_predictions})")
    print(f"XGBoost Model: {xgb_percentage_close:.2f}% of the time, the actual value is >= {threshold_num} when the prediction is >= {threshold_num} (Number of such predictions: {xgb_num_predictions})")



Linear Model: 23.53% of the time, the actual value is >= 0.2 when the prediction is >= 0.2 (Number of such predictions: 136)
XGBoost Model: 33.33% of the time, the actual value is >= 0.2 when the prediction is >= 0.2 (Number of such predictions: 198)


Linear Model: 14.29% of the time, the actual value is >= 0.3 when the prediction is >= 0.3 (Number of such predictions: 98)
XGBoost Model: 28.12% of the time, the actual value is >= 0.3 when the prediction is >= 0.3 (Number of such predictions: 160)


Linear Model: 4.62% of the time, the actual value is >= 0.4 when the prediction is >= 0.4 (Number of such predictions: 65)
XGBoost Model: 25.76% of the time, the actual value is >= 0.4 when the prediction is >= 0.4 (Number of such predictions: 132)


Linear Model: 2.27% of the time, the actual value is >= 0.5 when the prediction is >= 0.5 (Number of such predictions: 44)
XGBoost Model: 33.33% of the time, the actual value is >= 0.5 when the prediction is >= 0.5 (Number of such predictions

### GET FTD DATA

In [267]:
sesh = requests.session()
sesh.headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "en-US,en;q=0.9",
    "Cache-Control": "max-age=0",
    "Priority": "u=0, i",
    "Referer": "https://www.google.com/",
    "Sec-Ch-Ua": '"Not/A)Brand";v="8", "Chromium";v="126", "Google Chrome";v="126"',
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Ch-Ua-Platform": '"Windows"',
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "cross-site",
    "Sec-Fetch-User": "?1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36"
}

# URL of the website
url = 'https://www.sec.gov/data/foiadocsfailsdatahtm'

# Send a GET request to the website
response = sesh.get(url)
print(response.status_code)

200


In [269]:
# Parse the HTML content
soup = BeautifulSoup(response.text, 'html.parser')

# Find all <a> tags
anchor_tags = soup.find_all('a')

# Filter hrefs containing 'fails-deliver-data'
href_list = [a['href'] for a in anchor_tags if 'fails-deliver-data' in a.get('href', '')]

In [13]:
# Download each file
for href in href_list:
    file_url = f"https://www.sec.gov{href}"
    file_name = os.path.join('FTD_Downloads', os.path.basename(href))
    
    # Send a GET request to download the file
    file_response = sesh.get(file_url)
    
    # Check if the request was successful
    if file_response.status_code == 200:
        # Check if the content type is appropriate for the file
        content_type = file_response.headers.get('Content-Type', '')
        if 'application' in content_type:
            with open(file_name, 'wb') as file:
                file.write(file_response.content)
            print(f"Downloaded: {file_name}")
        else:
            print(f"Skipping invalid content type for {file_name}: {content_type}")
    else:
        print(f"Failed to download {file_name}: Status code {file_response.status_code}")


Downloaded: FTD_Downloads\cnsfails202405b.zip
Downloaded: FTD_Downloads\cnsfails202405a.zip
Downloaded: FTD_Downloads\cnsfails202404b.zip
Downloaded: FTD_Downloads\cnsfails202404a.zip
Downloaded: FTD_Downloads\cnsfails202403b.zip
Downloaded: FTD_Downloads\cnsfails202403a.zip
Downloaded: FTD_Downloads\cnsfails202402b.zip
Downloaded: FTD_Downloads\cnsfails202402a.zip
Downloaded: FTD_Downloads\cnsfails202401b.zip
Downloaded: FTD_Downloads\cnsfails202401a.zip
Downloaded: FTD_Downloads\cnsfails202312b.zip
Downloaded: FTD_Downloads\cnsfails202312a.zip
Downloaded: FTD_Downloads\cnsfails202311b.zip
Downloaded: FTD_Downloads\cnsfails202311a.zip
Downloaded: FTD_Downloads\cnsfails202310b.zip
Downloaded: FTD_Downloads\cnsfails202310a.zip
Downloaded: FTD_Downloads\cnsfails202309b.zip
Downloaded: FTD_Downloads\cnsfails202309a.zip
Downloaded: FTD_Downloads\cnsfails202308b.zip
Downloaded: FTD_Downloads\cnsfails202308a.zip
Downloaded: FTD_Downloads\cnsfails202307b.zip
Downloaded: FTD_Downloads\cnsfails

In [270]:
# Create output directory if it doesn't exist
output_dir = 'FTD_CSVs'

# Directory containing the ZIP files
zip_dir = 'FTD_Downloads'

In [364]:
# Loop through all ZIP files in the directory
for zip_filename in os.listdir(zip_dir):
    if zip_filename.endswith('.zip'):
        zip_path = os.path.join(zip_dir, zip_filename)
        
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            # Extract all files from the ZIP archive
            zip_ref.extractall(zip_dir)
            
            # Loop through the extracted files
            for extracted_file in zip_ref.namelist():
                if extracted_file.endswith('.txt') or '.' not in os.path.basename(extracted_file):
                    txt_file_path = os.path.join(zip_dir, extracted_file)
                    
                    try:
                        # Read the text file into a DataFrame
                        try:
                            df = pd.read_csv(txt_file_path, sep='|', header=0, on_bad_lines='warn')
                        except Exception:
                            df = pd.read_csv(txt_file_path, sep='|', header=0, on_bad_lines='warn', encoding='latin1')
                        
                        # Convert the DataFrame to a CSV file
                        csv_filename = os.path.splitext(os.path.basename(extracted_file))[0] + '.csv'
                        csv_file_path = os.path.join(output_dir, csv_filename)
                        df.to_csv(csv_file_path, index=False)
                        
                        print(f"Converted {txt_file_path} to {csv_file_path}")
                    except Exception as e:
                        print(f'Unable to convert file: {txt_file_path}. Error: {str(e)}')
                    
print("All files processed.")


Converted FTD_Downloads\cnsfails200907a.txt to FTD_CSVs\cnsfails200907a.csv
Converted FTD_Downloads\cnsfails200907b.txt to FTD_CSVs\cnsfails200907b.csv
Converted FTD_Downloads\cnsfails200908a.txt to FTD_CSVs\cnsfails200908a.csv
Converted FTD_Downloads\cnsfails200908b.txt to FTD_CSVs\cnsfails200908b.csv
Converted FTD_Downloads\cnsfails200909a.txt to FTD_CSVs\cnsfails200909a.csv
Converted FTD_Downloads\cnsfails200909b.txt to FTD_CSVs\cnsfails200909b.csv
Converted FTD_Downloads\cnsfails200910a.txt to FTD_CSVs\cnsfails200910a.csv
Converted FTD_Downloads\cnsfails200910b.txt to FTD_CSVs\cnsfails200910b.csv
Converted FTD_Downloads\cnsfails200911a.txt to FTD_CSVs\cnsfails200911a.csv
Converted FTD_Downloads\cnsfails200911b.txt to FTD_CSVs\cnsfails200911b.csv
Converted FTD_Downloads\cnsfails200912a.txt to FTD_CSVs\cnsfails200912a.csv
Converted FTD_Downloads\cnsfails200912b.txt to FTD_CSVs\cnsfails200912b.csv
Converted FTD_Downloads\cnsfails201001a.txt to FTD_CSVs\cnsfails201001a.csv
Converted FT

b'Skipping line 52725: expected 6 fields, saw 7\nSkipping line 58604: expected 6 fields, saw 7\n'


Converted FTD_Downloads\cnsfails202104b.txt to FTD_CSVs\cnsfails202104b.csv
Converted FTD_Downloads\cnsfails202105a.txt to FTD_CSVs\cnsfails202105a.csv
Converted FTD_Downloads\cnsfails202105b.txt to FTD_CSVs\cnsfails202105b.csv
Converted FTD_Downloads\cnsfails202106a.txt to FTD_CSVs\cnsfails202106a.csv
Converted FTD_Downloads\cnsfails202106b.txt to FTD_CSVs\cnsfails202106b.csv
Converted FTD_Downloads\cnsfails202107a.txt to FTD_CSVs\cnsfails202107a.csv
Converted FTD_Downloads\cnsfails202107b.txt to FTD_CSVs\cnsfails202107b.csv
Converted FTD_Downloads\cnsfails202108a.txt to FTD_CSVs\cnsfails202108a.csv
Converted FTD_Downloads\cnsfails202108b.txt to FTD_CSVs\cnsfails202108b.csv
Converted FTD_Downloads\cnsfails202109a.txt to FTD_CSVs\cnsfails202109a.csv
Converted FTD_Downloads\cnsfails202109b.txt to FTD_CSVs\cnsfails202109b.csv
Converted FTD_Downloads\cnsfails202110a.txt to FTD_CSVs\cnsfails202110a.csv
Converted FTD_Downloads\cnsfails202110b.txt to FTD_CSVs\cnsfails202110b.csv
Converted FT

b'Skipping line 286: expected 6 fields, saw 7\nSkipping line 356: expected 6 fields, saw 7\nSkipping line 2857: expected 6 fields, saw 7\nSkipping line 5504: expected 6 fields, saw 7\nSkipping line 8160: expected 6 fields, saw 7\nSkipping line 8249: expected 6 fields, saw 7\nSkipping line 10937: expected 6 fields, saw 7\nSkipping line 11017: expected 6 fields, saw 7\nSkipping line 13610: expected 6 fields, saw 7\nSkipping line 13694: expected 6 fields, saw 7\nSkipping line 16259: expected 6 fields, saw 7\nSkipping line 16338: expected 6 fields, saw 7\nSkipping line 18919: expected 6 fields, saw 7\nSkipping line 18997: expected 6 fields, saw 7\nSkipping line 21575: expected 6 fields, saw 7\nSkipping line 21655: expected 6 fields, saw 7\nSkipping line 22148: expected 6 fields, saw 7\nSkipping line 24209: expected 6 fields, saw 7\nSkipping line 24284: expected 6 fields, saw 7\nSkipping line 24762: expected 6 fields, saw 7\nSkipping line 26855: expected 6 fields, saw 7\nSkipping line 26937

Converted FTD_Downloads\cnsp_sec_fails_200408.txt to FTD_CSVs\cnsp_sec_fails_200408.csv
Converted FTD_Downloads\cnsp_sec_fails_200410.txt to FTD_CSVs\cnsp_sec_fails_200410.csv


...

Converted FTD_Downloads\cnsp_sec_fails_200412.txt to FTD_CSVs\cnsp_sec_fails_200412.csv


b'Skipping line 267: expected 6 fields, saw 7\nSkipping line 2897: expected 6 fields, saw 7\nSkipping line 5512: expected 6 fields, saw 7\nSkipping line 8162: expected 6 fields, saw 7\nSkipping line 8243: expected 6 fields, saw 7\nSkipping line 10801: expected 6 fields, saw 7\nSkipping line 10884: expected 6 fields, saw 7\nSkipping line 13432: expected 6 fields, saw 7\nSkipping line 16141: expected 6 fields, saw 7\nSkipping line 18861: expected 6 fields, saw 7\nSkipping line 21457: expected 6 fields, saw 7\nSkipping line 24306: expected 6 fields, saw 7\nSkipping line 27079: expected 6 fields, saw 7\nSkipping line 29794: expected 6 fields, saw 7\nSkipping line 32551: expected 6 fields, saw 7\nSkipping line 35349: expected 6 fields, saw 7\nSkipping line 38030: expected 6 fields, saw 7\nSkipping line 40769: expected 6 fields, saw 7\nSkipping line 43406: expected 6 fields, saw 7\nSkipping line 43966: expected 6 fields, saw 7\nSkipping line 46167: expected 6 fields, saw 7\n'
b'Skipping line

Converted FTD_Downloads\cnsp_sec_fails_200411.txt to FTD_CSVs\cnsp_sec_fails_200411.csv
Converted FTD_Downloads\cnsp_sec_fails_200501.txt to FTD_CSVs\cnsp_sec_fails_200501.csv
Converted FTD_Downloads\cnsp_sec_fails_200503.txt to FTD_CSVs\cnsp_sec_fails_200503.csv
Converted FTD_Downloads\cnsp_sec_fails_200502.txt to FTD_CSVs\cnsp_sec_fails_200502.csv
Converted FTD_Downloads\cnsp_sec_fails_200506.txt to FTD_CSVs\cnsp_sec_fails_200506.csv
Converted FTD_Downloads\cnsp_sec_fails_200504.txt to FTD_CSVs\cnsp_sec_fails_200504.csv
Converted FTD_Downloads\cnsp_sec_fails_200505.txt to FTD_CSVs\cnsp_sec_fails_200505.csv
Converted FTD_Downloads\cnsp_sec_fails_200508.txt to FTD_CSVs\cnsp_sec_fails_200508.csv
Converted FTD_Downloads\cnsp_sec_fails_200509.txt to FTD_CSVs\cnsp_sec_fails_200509.csv
Converted FTD_Downloads\cnsp_sec_fails_200507.txt to FTD_CSVs\cnsp_sec_fails_200507.csv
Converted FTD_Downloads\cnsp_sec_fails_200512.txt to FTD_CSVs\cnsp_sec_fails_200512.csv
Converted FTD_Downloads\cnsp_sec

c:\ProgramData\Anaconda3\envs\py37\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Converted FTD_Downloads\cnsp_sec_fails_200809.txt to FTD_CSVs\cnsp_sec_fails_200809.csv
Converted FTD_Downloads\cnsp_sec_fails_200808.txt to FTD_CSVs\cnsp_sec_fails_200808.csv
Converted FTD_Downloads\cnsp_sec_fails_200807.txt to FTD_CSVs\cnsp_sec_fails_200807.csv
Converted FTD_Downloads\cnsp_sec_fails_200811.txt to FTD_CSVs\cnsp_sec_fails_200811.csv
Converted FTD_Downloads\cnsp_sec_fails_200810.txt to FTD_CSVs\cnsp_sec_fails_200810.csv
Converted FTD_Downloads\cnsp_sec_fails_200812.txt to FTD_CSVs\cnsp_sec_fails_200812.csv
Converted FTD_Downloads\cnsp_sec_fails_200901.txt to FTD_CSVs\cnsp_sec_fails_200901.csv
Converted FTD_Downloads\cnsp_sec_fails_200903.txt to FTD_CSVs\cnsp_sec_fails_200903.csv
Converted FTD_Downloads\cnsp_sec_fails_200902.txt to FTD_CSVs\cnsp_sec_fails_200902.csv
Converted FTD_Downloads\cnsp_sec_fails_200905.txt to FTD_CSVs\cnsp_sec_fails_200905.csv
Converted FTD_Downloads\cnsp_sec_fails_200904.txt to FTD_CSVs\cnsp_sec_fails_200904.csv
Converted FTD_Downloads\cnsp_sec

#### COMBINE FTD DATA FILES

In [366]:
dfs = []
folder_path = 'FTD_CSVs'
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        dfs.append(pd.read_csv(file_path, index_col=None))

df = pd.concat(dfs)
df.to_csv('ftd_all_data_6_19_24.csv')

df = df[df['SYMBOL'].isin(gme_tickers)].reset_index(drop=True)
print(len(df['SYMBOL'].unique().tolist()))
df.to_csv('ftds_gme_related_6_19_24.csv')

c:\ProgramData\Anaconda3\envs\py37\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


87


### Get All GME & GME ETF Historical Prices

In [296]:
ftd_df = pd.read_csv('ftds_gme_related_6_19_24.csv', index_col=0)
gme_tickers = ftd_df['ticker'].unique().tolist()
print(len(gme_tickers))
tdfs = []
for tick in gme_tickers:
        try:
            # Define the ticker symbol
            ticker_symbol = tick

            # Get data on this ticker
            ticker_data = yf.Ticker(ticker_symbol)

            # Get the historical prices for this ticker
            ticker_df = ticker_data.history(period='1d', start='2004-01-01', end='2024-06-19')

            ticker_df['Date'] = ticker_df.index
            ticker_df.reset_index(drop=True, inplace=True)
            ticker_df['Date'] = pd.to_datetime(ticker_df['Date'])
            ticker_df['Date'] = ticker_df['Date'].dt.date
            ticker_df['Ticker'] = ticker_symbol
            tdfs.append(ticker_df)

            # Save the data to a CSV file
            ticker_df.to_csv(f'historical_prices/historical_{ticker_symbol}_prices.csv')
            print(f'Ticker: {ticker_symbol} Data Recorded')
        except:
              print(f'No Data For: {ticker_symbol}')

In [297]:
all_ticker_df = pd.concat(tdfs)

col_sort = ['Date', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits', 'Capital Gains']
all_ticker_df = all_ticker_df[col_sort]

new_cols = ['date', 'ticker', 'open', 'high', 'low', 'close', 'volume', 'dividends', 'stock_splits', 'capital_gains']
all_ticker_df.columns = new_cols

all_ticker_df.to_csv('all_gme_related_prices.csv')